## Data cleaning and Preprocessing

In [1]:
# Importing modules

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

#### Cleaning sales file

In [2]:
# Read the sales file

sales = pd.read_csv("C:/Users/spand/Projects/DATASPARK/Unclean datasets/Sales.csv")
sales.head()

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD


In [3]:
# Checking data types
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order Number   62884 non-null  int64 
 1   Line Item      62884 non-null  int64 
 2   Order Date     62884 non-null  object
 3   Delivery Date  13165 non-null  object
 4   CustomerKey    62884 non-null  int64 
 5   StoreKey       62884 non-null  int64 
 6   ProductKey     62884 non-null  int64 
 7   Quantity       62884 non-null  int64 
 8   Currency Code  62884 non-null  object
dtypes: int64(6), object(3)
memory usage: 4.3+ MB


In [4]:
# Checking for null values
sales.isnull().sum()

Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64

In [5]:
# Handling null values

# Filling null values of Delivery Date with the corresponding Order Date assuming that the product was bought in-store
sales.loc[sales["Delivery Date"].isna() & (sales["StoreKey"] != 0), "Delivery Date"] = (sales["Order Date"])

In [6]:
# Convert Currency Code to string data type & Order Date and Delivery Date to datetime format

sales["Currency Code"] = sales["Currency Code"].astype(str)
sales["Order Date"] = pd.to_datetime(sales["Order Date"], format="%m/%d/%Y")
sales["Delivery Date"] = pd.to_datetime(sales["Delivery Date"], format="%m/%d/%Y", errors="coerce")

In [7]:
# Adding a Frequency column to later perform EDA

# Count the number of orders per customer
order_counts = sales.groupby("CustomerKey")["Order Number"].count()

# Function to categorize the frequency
def frequency(count):
    if count >= 10:
        return "Frequent Buyer"
    elif count >= 5 and count < 10:
        return "Moderate Buyer"
    elif count >= 2 and count < 5:
        return "Occasional Buyer"
    else:
        return "New Buyer"

# Apply the frequency function
order_counts_category = order_counts.apply(frequency).reset_index(name="Frequency")

# Merge frequency category back to sales
sales = pd.merge(sales, order_counts_category, on="CustomerKey", how="left")

In [8]:
sales

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code,Frequency
0,366000,1,2016-01-01,2016-01-01,265598,10,1304,1,CAD,Moderate Buyer
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,Occasional Buyer
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,Occasional Buyer
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,Moderate Buyer
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,Moderate Buyer
...,...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2021-02-20,2021-02-20,1216913,43,632,3,USD,Frequent Buyer
62880,2243031,1,2021-02-20,2021-02-24,511229,0,98,4,EUR,Occasional Buyer
62881,2243032,1,2021-02-20,2021-02-23,331277,0,1613,2,CAD,Moderate Buyer
62882,2243032,2,2021-02-20,2021-02-23,331277,0,1717,2,CAD,Moderate Buyer


In [9]:
# Converting cleaned dataframe into a csv
sales.to_csv(r"C:\Users\spand\Projects\DATASPARK\Clean datasets\Sales_C.csv", index=False)

#### Cleaning customers file

In [10]:
# Read the customers file

customers = pd.read_csv("C:/Users/spand/Projects/DATASPARK/Unclean datasets/Customers.csv", encoding="ISO-8859-1")
customers.head()

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965


In [11]:
# Checking data types
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerKey  15266 non-null  int64 
 1   Gender       15266 non-null  object
 2   Name         15266 non-null  object
 3   City         15266 non-null  object
 4   State Code   15256 non-null  object
 5   State        15266 non-null  object
 6   Zip Code     15266 non-null  object
 7   Country      15266 non-null  object
 8   Continent    15266 non-null  object
 9   Birthday     15266 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.2+ MB


In [12]:
# Checking for null values
customers.isnull().sum()

CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [13]:
# Handling null values

# Seeing what the null values are
customers[customers.isnull().any(axis=1)]

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
5304,729681,Female,Rossana Padovesi,Polvica,NaN,Napoli,80035,Italy,Europe,4/18/1981
5316,732289,Male,Indro Piccio,Varcaturo,NaN,Napoli,80014,Italy,Europe,2/24/1949
5372,742042,Male,Amaranto Loggia,Casaferro,NaN,Napoli,80034,Italy,Europe,3/14/1936
5377,742886,Female,Edmonda Capon,Terzigno,NaN,Napoli,80040,Italy,Europe,8/6/1963
5378,743343,Female,Ambra Sagese,Pomigliano D'Arco,NaN,Napoli,80038,Italy,Europe,1/5/1961
5485,759705,Male,Callisto Lo Duca,Casilli,NaN,Napoli,80047,Italy,Europe,8/28/1976
5525,765589,Male,Michelino Lucchesi,Pompei Scavi,NaN,Napoli,80045,Italy,Europe,11/13/1947
5531,766410,Male,Adelmio Beneventi,Licola,NaN,Napoli,80078,Italy,Europe,1/13/1940
5631,781667,Female,Ilda Manna,Napoli,NaN,Napoli,80134,Italy,Europe,5/8/1977
5695,789177,Male,Calogero Folliero,Mariglianella,NaN,Napoli,80030,Italy,Europe,3/3/2000


In [14]:
# Replacing State Code null values as NA as it is the state code for Napoli
customers["State Code"] = customers["State Code"].replace(np.nan, "NA")

In [15]:
# Renaming state and country columns
customers.rename(columns={"State": "C_State", "Country": "C_Country"}, inplace=True)

In [16]:
# Convert Birthday to Datetime format
customers["Birthday"] = pd.to_datetime(customers["Birthday"], format="%m/%d/%Y", errors="coerce")
customers["Birthday"]

0       1939-07-03
1       1979-09-27
2       1947-05-26
3       1957-09-17
4       1965-11-19
           ...    
15261   1936-03-25
15262   1992-02-16
15263   1937-11-09
15264   1937-10-12
15265   1965-08-18
Name: Birthday, Length: 15266, dtype: datetime64[ns]

In [17]:
# Add columns Age and Age Category from Birthday that can later be used for EDA

# Calculate Age
customers["Age"] = sales["Order Date"].dt.year - customers["Birthday"].dt.year

# Age Category Function
def categorize_age(age):
    if age < 18:
        return "Minors"
    elif 18 <= age <= 30:
        return "Youth"
    elif 30 <= age <= 60:
        return "Adults"
    else:
        return "Senior Citizens"

# Apply Age Category
customers["Age Category"] = customers["Age"].apply(categorize_age)
customers["Age"] = customers["Age"].astype(int)

In [18]:
customers

,CustomerKey,Gender,Name,City,State Code,C_State,Zip Code,C_Country,Continent,Birthday,Age,Age Category
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03,77,Senior Citizens
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27,37,Adults
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26,69,Senior Citizens
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17,59,Adults
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19,51,Adults
...,...,...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,1936-03-25,82,Senior Citizens
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,1992-02-16,26,Youth
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,1937-11-09,81,Senior Citizens
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,1937-10-12,81,Senior Citizens


In [19]:
# Converting cleaned dataframe into a csv
customers.to_csv(r"C:\Users\spand\Projects\DATASPARK\Clean datasets\Customers_C.csv", index=False)

#### Cleaning exchange_rates file

In [20]:
# Read the exchange_rates file

exchange_rates = pd.read_csv("C:/Users/spand/Projects/DATASPARK/Unclean datasets/Exchange_Rates.csv")
exchange_rates.head()

,Date,Currency,Exchange
0,1/1/2015,USD,1.0000
1,1/1/2015,CAD,1.1583
2,1/1/2015,AUD,1.2214
3,1/1/2015,EUR,0.8237
4,1/1/2015,GBP,0.6415


In [21]:
# Checking data types
exchange_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11215 entries, 0 to 11214
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      11215 non-null  object 
 1   Currency  11215 non-null  object 
 2   Exchange  11215 non-null  float64
dtypes: float64(1), object(2)
memory usage: 263.0+ KB


In [22]:
# Checking for null values
exchange_rates.isnull().sum()

Date        0
Currency    0
Exchange    0
dtype: int64

In [23]:
# Convert Date to Datetime format
exchange_rates["Date"] = pd.to_datetime(exchange_rates["Date"], format="%m/%d/%Y", errors="coerce")
exchange_rates["Date"]

0       2015-01-01
1       2015-01-01
2       2015-01-01
3       2015-01-01
4       2015-01-01
           ...    
11210   2021-02-20
11211   2021-02-20
11212   2021-02-20
11213   2021-02-20
11214   2021-02-20
Name: Date, Length: 11215, dtype: datetime64[ns]

In [24]:
exchange_rates

,Date,Currency,Exchange
0,2015-01-01,USD,1.0000
1,2015-01-01,CAD,1.1583
2,2015-01-01,AUD,1.2214
3,2015-01-01,EUR,0.8237
4,2015-01-01,GBP,0.6415
...,...,...,...
11210,2021-02-20,USD,1.0000
11211,2021-02-20,CAD,1.2610
11212,2021-02-20,AUD,1.2723
11213,2021-02-20,EUR,0.8238


In [25]:
# Converting cleaned dataframe into a csv
exchange_rates.to_csv(r"C:\Users\spand\Projects\DATASPARK\Clean datasets\Exchange_Rates_C.csv", index=False)

#### Cleaning products file

In [26]:
# Read the products file

products = pd.read_csv("C:/Users/spand/Projects/DATASPARK/Unclean datasets/Products.csv")
products.head()

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio


In [27]:
# Checking data types
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ProductKey      2517 non-null   int64 
 1   Product Name    2517 non-null   object
 2   Brand           2517 non-null   object
 3   Color           2517 non-null   object
 4   Unit Cost USD   2517 non-null   object
 5   Unit Price USD  2517 non-null   object
 6   SubcategoryKey  2517 non-null   int64 
 7   Subcategory     2517 non-null   object
 8   CategoryKey     2517 non-null   int64 
 9   Category        2517 non-null   object
dtypes: int64(3), object(7)
memory usage: 196.8+ KB


In [28]:
# Checking for null values
products.isnull().sum()

ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64

In [29]:
# Convert Unit Cost USD to float data type and remove $
products["Unit Cost USD"] = (products["Unit Cost USD"].replace(r"[\$,]", "", regex=True).astype(float))
products["Unit Cost USD"]

0        6.62
1        6.62
2        7.40
3       11.00
4       11.00
        ...  
2512    43.07
2513    43.07
2514     1.71
2515     1.71
2516     1.71
Name: Unit Cost USD, Length: 2517, dtype: float64

In [30]:
# Convert Unit Price USD to float data type and remove $
products["Unit Price USD"] = (products["Unit Price USD"].replace(r"[\$,]", "", regex=True).astype(float))
products["Unit Price USD"]

0        12.99
1        12.99
2        14.52
3        21.57
4        21.57
         ...  
2512    129.99
2513    129.99
2514      3.35
2515      3.35
2516      3.35
Name: Unit Price USD, Length: 2517, dtype: float64

In [31]:
# Add columns Revenue and Profit that can later be used for EDA

# Calculating Reveunue Per Customer USD and Profit Per CustomerUSD
products["Revenue USD"] = sales["Quantity"] * products["Unit Price USD"]
products["Profit USD"] = products["Revenue USD"] - (sales["Quantity"] * products["Unit Cost USD"])

In [32]:
products

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category,Revenue USD,Profit USD
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,101,MP4&MP3,1,Audio,12.99,6.37
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,101,MP4&MP3,1,Audio,25.98,12.74
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,101,MP4&MP3,1,Audio,14.52,7.12
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,101,MP4&MP3,1,Audio,150.99,73.99
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,101,MP4&MP3,1,Audio,21.57,10.57
...,...,...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,43.07,129.99,505,Cell phones Accessories,5,Cell phones,259.98,173.84
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,43.07,129.99,505,Cell phones Accessories,5,Cell phones,129.99,86.92
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,1.71,3.35,505,Cell phones Accessories,5,Cell phones,10.05,4.92
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,1.71,3.35,505,Cell phones Accessories,5,Cell phones,3.35,1.64


In [33]:
# Converting cleaned dataframe into a csv
products.to_csv(r"C:\Users\spand\Projects\DATASPARK\Clean datasets\Products_C.csv", index=False)

#### Cleaning stores file

In [34]:
# Read the stores file

stores = pd.read_csv("C:/Users/spand/Projects/DATASPARK/Unclean datasets/Stores.csv")
stores.head()

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015


In [35]:
# Checking data types
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   StoreKey       67 non-null     int64  
 1   Country        67 non-null     object 
 2   State          67 non-null     object 
 3   Square Meters  66 non-null     float64
 4   Open Date      67 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.7+ KB


In [36]:
# Checking for null values
stores.isnull().sum()

StoreKey         0
Country          0
State            0
Square Meters    1
Open Date        0
dtype: int64

In [37]:
# Handling null values

# Seeing what the null values are
stores[stores.isnull().any(axis=1)]

,StoreKey,Country,State,Square Meters,Open Date
66,0,Online,Online,NaN,1/1/2010


In [38]:
# Replacing null values with 0 as there is no square meters for an online store
stores.fillna(0, inplace=True)

In [39]:
# Renaming state and country columns
stores.rename(columns={"State": "S_State", "Country": "S_Country"}, inplace=True)

In [40]:
# Convert Open Date to Datetime format
stores["Open Date"] = pd.to_datetime(stores["Open Date"], format="%m/%d/%Y", errors = "coerce")
stores["Open Date"]

0    2008-01-01
1    2008-01-12
2    2012-01-07
3    2010-01-01
4    2015-12-09
        ...    
62   2008-03-06
63   2010-01-01
64   2012-01-01
65   2014-01-01
66   2010-01-01
Name: Open Date, Length: 67, dtype: datetime64[ns]

In [41]:
# Add column Store Age that can later be used for EDA

# Calculate Store Age based on Order Date
stores["Store Age"] = abs(stores["Open Date"].dt.year - sales["Order Date"].dt.year)
stores["Store Age"] = stores["Store Age"].astype(int)

In [42]:
stores

,StoreKey,S_Country,S_State,Square Meters,Open Date,Store Age
0,1,Australia,Australian Capital Territory,595.0,2008-01-01,8
1,2,Australia,Northern Territory,665.0,2008-01-12,8
2,3,Australia,South Australia,2000.0,2012-01-07,4
3,4,Australia,Tasmania,2000.0,2010-01-01,6
4,5,Australia,Victoria,2000.0,2015-12-09,1
...,...,...,...,...,...,...
62,63,United States,Utah,2000.0,2008-03-06,8
63,64,United States,Washington DC,1330.0,2010-01-01,6
64,65,United States,West Virginia,1785.0,2012-01-01,4
65,66,United States,Wyoming,840.0,2014-01-01,2


In [43]:
# Converting cleaned dataframe into a csv
stores.to_csv(r"C:\Users\spand\Projects\DATASPARK\Clean datasets\Stores_C.csv", index=False)

## Merging cleaned dataframes to perform EDA

In [44]:
# Add sales data to df
Global_electronics_data = sales

In [45]:
# Merge stores data to df
Global_electronics_data = pd.merge(Global_electronics_data, stores, on="StoreKey", how="left")

In [46]:
# Merge productus data to df
Global_electronics_data = pd.merge(Global_electronics_data, products, on="ProductKey", how="left")

In [47]:
# Merge customers data to df
Global_electronics_data = pd.merge(Global_electronics_data, customers, on="CustomerKey", how="left")

In [48]:
# Merge exchange_rates data to df
Global_electronics_data = pd.merge(Global_electronics_data, exchange_rates, left_on=["Order Date", "Currency Code"], right_on=["Date", "Currency"], how="left")

In [49]:
# Drop unnecessary columns to avoid repetitive values
Global_electronics_data.drop(columns=["Date", "Currency"], inplace=True)

In [50]:
# Replacing spaces in column names with underscore
Global_electronics_data.columns = Global_electronics_data.columns.str.replace(" ", "_")

In [51]:
Global_electronics_data.head(5)

,Order_Number,Line_Item,Order_Date,Delivery_Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency_Code,Frequency,...,City,State_Code,C_State,Zip_Code,C_Country,Continent,Birthday,Age,Age_Category,Exchange
0,366000,1,2016-01-01,2016-01-01,265598,10,1304,1,CAD,Moderate Buyer,...,London,ON,Ontario,N5W 5K6,Canada,North America,1971-03-23,45,Adults,1.3884
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,Occasional Buyer,...,Auberry,CA,California,93602,United States,North America,1995-11-20,22,Youth,1.0000
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,Occasional Buyer,...,Auberry,CA,California,93602,United States,North America,1995-11-20,22,Youth,1.0000
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,Moderate Buyer,...,Red Deer,AB,Alberta,T4N 2A6,Canada,North America,1962-08-24,54,Adults,1.3884
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,Moderate Buyer,...,Red Deer,AB,Alberta,T4N 2A6,Canada,North America,1962-08-24,54,Adults,1.3884


In [52]:
Global_electronics_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 38 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order_Number    62884 non-null  int64         
 1   Line_Item       62884 non-null  int64         
 2   Order_Date      62884 non-null  datetime64[ns]
 3   Delivery_Date   62884 non-null  datetime64[ns]
 4   CustomerKey     62884 non-null  int64         
 5   StoreKey        62884 non-null  int64         
 6   ProductKey      62884 non-null  int64         
 7   Quantity        62884 non-null  int64         
 8   Currency_Code   62884 non-null  object        
 9   Frequency       62884 non-null  object        
 10  S_Country       62884 non-null  object        
 11  S_State         62884 non-null  object        
 12  Square_Meters   62884 non-null  float64       
 13  Open_Date       62884 non-null  datetime64[ns]
 14  Store_Age       62884 non-null  int32         
 15  Pr

In [53]:
Global_electronics_data.isnull().sum()

Order_Number      0
Line_Item         0
Order_Date        0
Delivery_Date     0
CustomerKey       0
StoreKey          0
ProductKey        0
Quantity          0
Currency_Code     0
Frequency         0
S_Country         0
S_State           0
Square_Meters     0
Open_Date         0
Store_Age         0
Product_Name      0
Brand             0
Color             0
Unit_Cost_USD     0
Unit_Price_USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
Revenue_USD       0
Profit_USD        0
Gender            0
Name              0
City              0
State_Code        0
C_State           0
Zip_Code          0
C_Country         0
Continent         0
Birthday          0
Age               0
Age_Category      0
Exchange          0
dtype: int64

In [54]:
# Converting cleaned dataframe into a csv
Global_electronics_data.to_csv(r"C:\Users\spand\Projects\DATASPARK\Clean datasets\Global_electronics_data_C.csv", index=False)